In [ ]:
from dr import *

In [ ]:
# Set up device
import numpy as np
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Hyperparameters
num_items = 752
num_groups = 8
hidden_dim = 128
n_layers = 3
n_heads = 8
context_length = 20
max_timesteps = 200
dropout = 0.1
batch_size = 32
num_epochs = 10
learning_rate = 1e-4

# Initialize model
model = DecisionTransformer(
    num_items=num_items,
    num_groups=num_groups,
    hidden_dim=hidden_dim,
    n_layers=n_layers,
    n_heads=n_heads,
    contxt_len=context_length,
    max_timesteps=max_timesteps,
    dropout=dropout
).to(device)

print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")


Using device: cpu
Model parameters: 831,728


# Step 1: Load Training Data
Load trajectories from your dataset (Netflix, MovieLens, Goodreads, etc.)


In [ ]:
import pickle

with open('../data/processed/trajectories_train.pkl', 'rb') as f:
    trajectories = pickle.load(f)

# Step 2: Create Dataset and DataLoader


In [ ]:
# Create dataset
train_dataset = RecommendationDataset(trajectories=trajectories, context_length=context_length)

# Create dataloader
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=0
)

print(f"Dataset size: {len(train_dataset)}")
print(f"Batches per epoch: {len(train_loader)}")

# Test one batch
sample_batch = next(iter(train_loader))
print(f"\nSample batch shapes:")
for key, val in sample_batch.items():
    print(f"  {key}: {val.shape}")


NameError: name 'RecommendationDataset' is not defined

# Step 3: Set up Optimizer and Train


In [ ]:
# Create optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model using the train_decision_transformer function
trained_model = train_decision_transformer(
    model=model,
    train_loader=train_loader,
    optimizer=optimizer,
    device=device,
    num_epochs=num_epochs
)

print("Training complete!")


# Step 4: Inference Example
Make predictions with the trained model


In [ ]:
# Put model in eval mode
trained_model.eval()

# Get a test batch
test_batch = next(iter(train_loader))

# Move to device
states = test_batch['states'].to(device)
actions = test_batch['actions'].to(device)
rtg = test_batch['rtg'].to(device)
timesteps = test_batch['timesteps'].to(device)
groups = test_batch['groups'].to(device)

# Forward pass
with torch.no_grad():
    logits = trained_model(states, actions, rtg, timesteps, groups)

# Get predictions
predictions = torch.argmax(logits, dim=-1)

print(f"Logits shape: {logits.shape}")
print(f"Predictions shape: {predictions.shape}")
print(f"\nFirst 5 predictions from first sample in batch:")
print(predictions[0, :5].cpu().numpy())
print(f"\nTrue targets (first sample):")
print(test_batch['targets'][0, :5].numpy())
